In [269]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from scipy import stats


In [270]:
from keras.models import Model
from keras.layers import LSTM, Dropout, Dense, Conv1D, ConvLSTM2D, Input
from keras.optimizers import Adam, SGD,RMSprop
from numpy import save
from numpy import load
import sklearn
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [3]:
import pymongo
from pymongo import MongoClient
import pprint

import pandas as pd
import numpy as np
import subprocess

#bson to json
from bson.objectid import ObjectId
from bson.json_util import loads, dumps

import codecs
import sys
import json
import dns
import os

# Read Data

In [331]:
def read_data():  # returns a merged data file of features and target labels
    #The features are input
    
    df = pd.read_csv('Torso_Motion.txt', delimiter = ' ',dtype = 'float32')
    df.columns = ['Timestep','AccX','AccY','AccZ','Gyroscope_X','Gyroscope_Y','Gyroscope_Z','Mag_meter_X','Mag_meter_Y','Mag_meter_Z','Orientation_w','Orientation_x','Orientation_y','Orientation_z','Gravity_X','Gravity_Y','Gravity_Z','Linear_accX','Linear_accY','Linear_accZ','Pressure','Altitude','Temperature']
    
    d = pd.read_csv('Torso_Motion.txt', delimiter = ' ',dtype  = str,usecols=[0])
    d.columns = ['Timestep']
    df['Timestep'] = d['Timestep']
    df['Timestep']= df['Timestep'].apply(lambda x : x.split('.')[0])
    #return df
    #df.head()
    
    # input target labels
    
    dfLabel = pd.read_csv('Label.txt', delimiter=' ', dtype = 'float64')
    dfLabel.columns = ['Timestep','Coarse_label','Fine_label','Road','Traffic','Tunnel','Social','Food']
    
    dLabel = pd.read_csv('Label.txt', delimiter=' ', dtype = str, usecols=[0])
    dLabel.columns = ['Timestep']
    #dLabel.columns = ['Timestep','Coarse_label','Fine_label','Road','Traffic','Tunnel','Social','Food']
    
    dfLabel['Timestep'] = dLabel['Timestep']
    #dfLabel.head()
    #len(dfLabel)
    
    x = df
    y = dfLabel[['Timestep','Coarse_label','Fine_label']]
    
    data = pd.merge(x, y, on = 'Timestep')
    
    return data.dropna()

In [7]:
#data.head()

In [338]:
cd C:\Users\Nithish\Desktop

C:\Users\Nithish\Desktop


In [345]:
cd C:\Users\Nithish\Desktop\User1

C:\Users\Nithish\Desktop\User1


In [346]:
cd 270617/

C:\Users\Nithish\Desktop\User1\270617


# Data Preprocessing

In [11]:
def filter_data(data):
    
    data_filtered = data[(data['Coarse_label'] > 0  )].reset_index(drop = True) # removing claas labels zero
    
    #print(data_filtered['Coarse_label'].value_counts())
    
    return data_filtered
    

In [10]:
LEN_OF_SEQUENCE = 200
OVERLAP = 50 # 25% overlap
FEATURE_DIMENSION = 19

In [12]:
def make_numpy_matrix(data) :
    # This function converts the data into a matrix that could be fed inside lstm. Sequence len = 200
    
    X = []
    y = []
    for i in range(OVERLAP,len(data) - LEN_OF_SEQUENCE ,LEN_OF_SEQUENCE):
            #print(i)
        #label = 1.0

        accX = data['AccX'].values[i:i + LEN_OF_SEQUENCE]
        accX1 = data['AccX'].values[i - 50:i + LEN_OF_SEQUENCE- 50]
        accY = data['AccY'].values[i:i + LEN_OF_SEQUENCE]
        accY1 = data['AccY'].values[i - 50:i + LEN_OF_SEQUENCE- 50]
        accZ = data['AccZ'].values[i:i + LEN_OF_SEQUENCE]
        accZ1 = data['AccZ'].values[i - 50:i + LEN_OF_SEQUENCE- 50]

        Gyro_X = data['Gyroscope_X'].values[i :i + LEN_OF_SEQUENCE]
        Gyro_X1 = data['Gyroscope_X'].values[i - 50:i + LEN_OF_SEQUENCE- 50]
        Gyro_Y = data['Gyroscope_Y'].values[i :i + LEN_OF_SEQUENCE]
        Gyro_Y1 = data['Gyroscope_Y'].values[i - 50:i + LEN_OF_SEQUENCE- 50]
        Gyro_Z = data['Gyroscope_Z'].values[i :i + LEN_OF_SEQUENCE]
        Gyro_Z1 = data['Gyroscope_Z'].values[i - 50:i + LEN_OF_SEQUENCE- 50]

        Mag_X = data['Mag_meter_X'].values[i :i + LEN_OF_SEQUENCE]
        Mag_X1 = data['Mag_meter_X'].values[i - 50:i + LEN_OF_SEQUENCE- 50]
        Mag_Y = data['Mag_meter_Y'].values[i :i + LEN_OF_SEQUENCE]
        Mag_Y1 = data['Mag_meter_Y'].values[i - 50:i + LEN_OF_SEQUENCE- 50]
        Mag_Z = data['Mag_meter_Z'].values[i :i + LEN_OF_SEQUENCE]
        Mag_Z1 = data['Mag_meter_Z'].values[i - 50:i + LEN_OF_SEQUENCE- 50]

        Linear_X = data['Linear_accX'].values[i :i + LEN_OF_SEQUENCE]
        Linear_X1 = data['Linear_accX'].values[i - 50:i + LEN_OF_SEQUENCE- 50]
        Linear_Y = data['Linear_accY'].values[i :i + LEN_OF_SEQUENCE]
        Linear_Y1 = data['Linear_accY'].values[i - 50:i + LEN_OF_SEQUENCE- 50]
        Linear_Z = data['Linear_accZ'].values[i :i + LEN_OF_SEQUENCE]
        Linear_Z1 = data['Linear_accZ'].values[i - 50:i + LEN_OF_SEQUENCE- 50]

        Gravity_X = data['Gravity_X'].values[i :i + LEN_OF_SEQUENCE]
        Gravity_X1 = data['Gravity_X'].values[i - 50:i + LEN_OF_SEQUENCE- 50]
        Gravity_Y = data['Gravity_Y'].values[i :i + LEN_OF_SEQUENCE]
        Gravity_Y1 = data['Gravity_Y'].values[i - 50:i + LEN_OF_SEQUENCE- 50]
        Gravity_Z = data['Gravity_Z'].values[i :i + LEN_OF_SEQUENCE]
        Gravity_Z1 = data['Gravity_Z'].values[i - 50:i + LEN_OF_SEQUENCE- 50]

        Orient_X = data['Orientation_x'].values[i :i + LEN_OF_SEQUENCE]
        Orient_X1 = data['Orientation_x'].values[i - 50:i + LEN_OF_SEQUENCE- 50]
        Orient_W = data['Orientation_w'].values[i :i + LEN_OF_SEQUENCE]
        Orient_W1 = data['Orientation_w'].values[i - 50:i + LEN_OF_SEQUENCE- 50]
        Orient_Y = data['Orientation_y'].values[i :i + LEN_OF_SEQUENCE]
        Orient_Y1 = data['Orientation_y'].values[i - 50:i + LEN_OF_SEQUENCE- 50]
        Orient_Z = data['Orientation_z'].values[i :i + LEN_OF_SEQUENCE]
        Orient_Z1 = data['Orientation_z'].values[i - 50:i + LEN_OF_SEQUENCE- 50]

        X.append([accX,accY,accZ,Gyro_X,Gyro_Y,Gyro_Z,Mag_X,Mag_Y, Mag_Z,
                  Linear_X, Linear_Y, Linear_Z,Gravity_X, Gravity_Y, Gravity_Z
                 ,Orient_X, Orient_Y, Orient_Z,Orient_W])

        X.append([accX1,accY1,accZ1,Gyro_X1,Gyro_Y1,Gyro_Z1,Mag_X1,Mag_Y1, Mag_Z1,
                   Linear_X1, Linear_Y1, Linear_Z1,Gravity_X1, Gravity_Y1, Gravity_Z1
                 ,Orient_X1, Orient_Y1, Orient_Z1,Orient_W1])

        #Here we are taking the mode of labels in the time sequence instead of the last recorder activity

        labels = stats.mode(data['Coarse_label'][i :i + LEN_OF_SEQUENCE])
        y.append(labels[0][0])

        labels = stats.mode(data['Coarse_label'][i - 50:i + LEN_OF_SEQUENCE- 50])
        y.append(labels[0][0])

        #y.append(label)
        #y.append(label)
    X = np.asarray(X,dtype = 'float32')
    X  = X.reshape((-1,200,19))

    y = np.asarray(y, dtype = 'float32')
    
    return X,y



In [347]:
data = read_data()
filtered_data = filter_data(data)
X,y = make_numpy_matrix(filtered_data)

In [326]:
y.shape

(18254,)

In [327]:
df = pd.DataFrame(y, columns = ['label'])

In [291]:
df['label'].value_counts() ## Check distribution of data

6.0    9130
1.0    4246
4.0    3736
2.0    3617
3.0     105
Name: label, dtype: int64

## Save the processed data into .npy file

In [296]:
X.shape

(20834, 200, 19)

In [295]:
y.shape

(20834,)

In [268]:
cd C:\Users\Nithish\Desktop

C:\Users\Nithish\Desktop


In [372]:
cd C:\Users\Nithish\Desktop\User 1 Bag

C:\Users\Nithish\Desktop\User 1 Bag


In [349]:
save('1_Torso_27_6_X', X)

In [350]:
save('1_Torso_27_6_y',y)

In [373]:
dataX = load('1_Bag_27_6_X.npy')
datay = load('1_Bag_27_6_y.npy')
X = load('1_Bag_26_6_X.npy')
y = load('1_Bag_26_6_y.npy')
dataX, datay = concatenate(dataX, datay,X,y )

In [374]:
X = load('1_Bag_22_6_X.npy')
y = load('1_Bag_22_6_y.npy')

In [375]:
dataX, datay = concatenate(dataX, datay,X,y )

In [380]:
dataX.shape

(60882, 200, 19)

In [379]:
datay.shape

(60882,)

In [381]:
save('Bag_User1_fullX', dataX)
save('Bag_User1_fully', datay)

In [208]:
def concatenate(X,y, new_dataX, new_datay):
    
    
    dataX = np.concatenate((X, new_dataX), axis = 0)
    

  
    datay = np.concatenate((y, new_datay), axis = 0)
  #print(data.shape)

    return dataX, datay

In [84]:
cd SHLDataset_preview_v1_part3

C:\Users\Nithish\Desktop\SHLDataset_preview_v1_part3


In [85]:
cd SHLDataset_preview_v1/

C:\Users\Nithish\Desktop\SHLDataset_preview_v1_part3\SHLDataset_preview_v1


In [86]:
cd User3/

C:\Users\Nithish\Desktop\SHLDataset_preview_v1_part3\SHLDataset_preview_v1\User3


In [87]:
cd 140617/

C:\Users\Nithish\Desktop\SHLDataset_preview_v1_part3\SHLDataset_preview_v1\User3\140617


C:\Users\Nithish\Desktop\Research Internship\SHLDataset_preview_v1_part1\SHLDataset_preview_v1\User1\270617


In [33]:
filtered_data

,Timestep,AccX,AccY,AccZ,Gyroscope_X,Gyroscope_Y,Gyroscope_Z,Mag_meter_X,Mag_meter_Y,Mag_meter_Z,...,Gravity_Y,Gravity_Z,Linear_accX,Linear_accY,Linear_accZ,Pressure,Altitude,Temperature,Coarse_label,Fine_label
0,1498470339500,1.826706,-8.716733,1.270882,0.126351,-1.043744,-0.467572,0.161344,36.838909,7.990065,...,-8.949545,3.474233,-4.070962,-3.268496,-6.722267,1007.998291,0.0,0.0,2.0,5.0
1,1498470339510,1.664390,-9.048232,0.479703,-0.484148,-1.850490,-0.500799,0.714364,37.355968,7.321272,...,-9.001525,3.338862,-4.479416,-3.886921,-6.419862,1007.993713,0.0,0.0,2.0,5.0
2,1498470339520,0.616486,-9.157213,0.741797,-0.926514,-2.151571,-0.480267,1.034413,37.841473,6.840587,...,-9.023118,3.219236,-4.695837,-4.195946,-6.186209,1007.989136,0.0,0.0,2.0,5.0
3,1498470339530,0.327102,-9.599220,0.925615,-1.220342,-1.960565,-0.387256,1.355843,38.153896,6.557631,...,-9.108921,2.957080,-5.076182,-4.540824,-6.315186,1007.984558,0.0,0.0,2.0,5.0
4,1498470339540,0.235793,-10.012119,0.419633,-1.136532,-1.267445,-0.193769,1.656198,38.445248,6.007956,...,-9.147361,2.782432,-5.416662,-4.145084,-7.195375,1007.979919,0.0,0.0,2.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083501,1498508664860,-0.750136,-8.336208,-14.167012,-0.863798,-0.316319,-0.302846,2.984020,-4.382990,45.715466,...,-3.503667,-9.021282,0.079796,-0.587120,0.029067,1005.144409,0.0,0.0,4.0,8.0
2083502,1498508664870,0.064377,-6.847972,-12.598029,-0.989914,-0.379662,-0.365411,3.000000,-4.206099,45.574898,...,-3.429348,-9.060199,-0.136523,-1.144958,-1.680338,1005.133667,0.0,0.0,4.0,8.0
2083503,1498508664880,1.017720,-5.981620,-10.889175,-1.011755,-0.333938,-0.250309,2.647491,-4.187500,45.680004,...,-3.335444,-9.097939,-1.006741,-1.162032,-1.081365,1005.122986,0.0,0.0,4.0,8.0
2083504,1498508664890,3.302852,-6.285632,-9.333902,-0.994788,-0.246604,-0.180778,2.345878,-3.908378,45.966621,...,-3.259725,-9.125040,-0.058206,-1.405008,-0.646992,1005.112244,0.0,0.0,4.0,8.0


In [115]:
cd C:\\Users\\Nithish\\Desktop\\Research Internship

C:\Users\Nithish\Desktop\Research Internship


In [116]:
cd "Processed data"

C:\Users\Nithish\Desktop\Research Internship\Processed data


In [117]:
X = load('preview_26_X.npy')

In [120]:
y = load('preview_26_y.npy')

In [121]:
X.shape

(20834, 200, 19)

In [124]:
X

array([[[-1.287644, -0.396921,  1.051272, ...,  0.186443,  8.97094 ,
         11.245369],
        [ 7.211739,  3.382386,  1.348031, ...,  0.099049, -1.264701,
         -0.53442 ],
        [ 0.590349,  0.819364,  0.256122, ...,  0.908653,  0.717903,
          0.456225],
        ...,
        [ 0.79876 ,  0.8021  ,  0.803146, ...,  0.806277,  0.805268,
          0.799563],
        [ 0.795539,  0.789085,  0.78448 , ...,  0.690515,  0.682691,
          0.672951],
        [ 0.666277,  0.658535,  0.652566, ...,  0.626311,  0.627207,
          0.633954]],

       [[ 1.826706,  1.66439 ,  0.616486, ..., -2.741955, -2.994769,
         -3.349149],
        [-3.355531, -3.408008, -3.429466, ...,  3.662191,  4.374154,
          4.623731],
        [ 4.185286,  2.326051,  0.325581, ...,  2.177663,  2.42316 ,
          2.559636],
        ...,
        [ 0.612751,  0.615228,  0.624865, ...,  0.726042,  0.71982 ,
          0.71047 ],
        [ 0.701481,  0.685146,  0.674036, ...,  0.713296,  0.725149,
   

In [125]:
X_2d = X.reshape((20834,-1))

In [139]:
X_2d = X_2d.reshape((20834,200,19))

In [140]:
X_2d

array([[[-1.287644, -0.396921,  1.051272, ...,  0.186443,  8.97094 ,
         11.245369],
        [ 7.211739,  3.382386,  1.348031, ...,  0.099049, -1.264701,
         -0.53442 ],
        [ 0.590349,  0.819364,  0.256122, ...,  0.908653,  0.717903,
          0.456225],
        ...,
        [ 0.79876 ,  0.8021  ,  0.803146, ...,  0.806277,  0.805268,
          0.799563],
        [ 0.795539,  0.789085,  0.78448 , ...,  0.690515,  0.682691,
          0.672951],
        [ 0.666277,  0.658535,  0.652566, ...,  0.626311,  0.627207,
          0.633954]],

       [[ 1.826706,  1.66439 ,  0.616486, ..., -2.741955, -2.994769,
         -3.349149],
        [-3.355531, -3.408008, -3.429466, ...,  3.662191,  4.374154,
          4.623731],
        [ 4.185286,  2.326051,  0.325581, ...,  2.177663,  2.42316 ,
          2.559636],
        ...,
        [ 0.612751,  0.615228,  0.624865, ...,  0.726042,  0.71982 ,
          0.71047 ],
        [ 0.701481,  0.685146,  0.674036, ...,  0.713296,  0.725149,
   

In [141]:
X_2d.shape

(20834, 200, 19)

In [294]:
def feature_scaling(X):
    # Converting the 3d matrix to 2d for feature scaling
    X_2d = X.reshape((-1,19))
    
    scaler = MinMaxScaler(feature_range=(-1, 1))
    
    X_scaled = scaler.fit_transform(X_2d)
    
    # Rescaling the 2d matrix to 3d
    X_scaled_3d = X_scaled.reshape((-1,200,19))
    
    print(X_scaled_3d.shape)
    
    return(X_scaled_3d)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
def one_hot_target(y):
    
    y_one_hot = pd.get_dummies(y)
    
    return y_one_hot.values

In [233]:
#np.random.seed(32)
def shuffle_along_axis(a, axis): # This is for shuffling the dataset
    
    np.random.seed(32)
    
    idx = np.random.rand(*a.shape,).argsort(axis=axis)
    
    return np.take_along_axis(a,idx,axis=axis)


In [300]:
x = np.array([[-10,0.2],[10,0.5],[7,0.1]])
#feature_scaling(x)
scaler  = MinMaxScaler(feature_range=(-1,1))
scaler.fit_transform(x)

array([[-1. , -0.5],
       [ 1. ,  1. ],
       [ 0.7, -1. ]])

In [122]:
y.shape

(20834,)

In [113]:
from sklearn.preprocessing import MinMaxScaler

In [114]:
scaler = MinMaxScaler(feature_range=(-1, 1))

In [127]:
X_scaled = scaler.fit_transform(X_2d)

In [129]:
X_scaled.shape

(20834, 3800)

In [130]:
X_scaled_3d = X_scaled.reshape((20834,200,19))

In [131]:
X_scaled_3d.shape

(20834, 200, 19)

In [293]:
X_scaled_3d.round(5)

array([[[-5.6690e-02, -8.6800e-03,  9.3900e-02, ...,  2.5161e-01,
          5.5837e-01,  4.8395e-01],
        [ 3.0727e-01,  1.1028e-01, -4.8870e-02, ..., -6.1150e-02,
         -1.3233e-01,  6.0940e-02],
        [-9.6750e-02,  1.3000e-04,  3.9595e-01, ...,  3.2910e-02,
          1.9086e-01,  1.5162e-01],
        ...,
        [ 7.6923e-01,  7.6502e-01,  7.6179e-01, ...,  7.6790e-01,
          7.6337e-01,  7.4573e-01],
        [ 7.3206e-01,  7.1443e-01,  7.0353e-01, ...,  5.2413e-01,
          4.7968e-01,  4.4283e-01],
        [ 4.3839e-01,  4.3484e-01,  4.3308e-01, ...,  3.8755e-01,
          3.9076e-01,  4.0637e-01]],

       [[ 9.0730e-02,  1.0163e-01,  7.1360e-02, ...,  1.3489e-01,
         -1.3173e-01, -3.0334e-01],
        [-2.1323e-01, -2.2200e-01, -3.3599e-01, ...,  1.0843e-01,
          1.5366e-01,  3.1100e-01],
        [ 6.7140e-02,  8.3660e-02,  3.9818e-01, ...,  9.8810e-02,
          2.6675e-01,  2.4729e-01],
        ...,
        [ 3.5723e-01,  3.5380e-01,  3.7071e-01, ...,  

In [142]:
y_one_hot = pd.get_dummies(y)

In [148]:
y_one_hot

,1.0,2.0,3.0,4.0,6.0
0,0,1,0,0,0
1,0,1,0,0,0
2,0,1,0,0,0
3,0,1,0,0,0
4,0,1,0,0,0
...,...,...,...,...,...
20829,0,0,0,1,0
20830,0,0,0,1,0
20831,0,0,0,1,0
20832,0,0,0,1,0


In [243]:
a = [2,3,4,5,6,1]

In [247]:
b = [8,2,7,9,2,1]

In [252]:
a_

array([3, 4, 6, 1, 2, 5])

In [251]:
b_

array([2, 7, 2, 1, 8, 9])

In [248]:
b_ = shuffle_along_axis(np.array(b),0)

In [240]:
data_shuffled= shuffle_along_axis(data,0)

In [241]:
data_shuffled.shape

(41668, 200, 19)

In [242]:
data_shuffled

array([[[-9.557300e-01,  1.731032e+00,  2.868220e+00, ...,
          3.195282e+00, -1.130063e+00, -2.079500e-02],
        [ 3.474849e+00,  3.363592e+00,  7.517070e-01, ...,
         -4.156790e+00, -4.020957e+00,  4.746615e+00],
        [ 3.982551e+00,  2.061441e+00, -6.836210e-01, ...,
          9.213700e-01,  3.487770e-01, -3.391330e-01],
        ...,
        [ 1.826490e-01,  2.809540e-01,  5.427300e-02, ...,
          1.812560e-01,  2.164150e-01,  3.289500e-02],
        [ 4.586050e-01,  1.057990e-01,  6.373910e-01, ...,
          4.402020e-01,  7.180800e-02,  3.481100e-02],
        [ 2.000500e-02,  8.485200e-02,  1.097880e-01, ...,
          5.614700e-01,  4.002090e-01,  3.232200e-02]],

       [[-7.029290e-01,  1.133830e+00,  3.413307e+00, ...,
         -4.413240e-01, -5.998229e+00, -2.725360e-01],
        [ 5.800720e-01,  7.890560e-01, -1.714248e+00, ...,
         -1.091883e+00,  2.484029e+00, -1.400312e+00],
        [ 2.781267e+00,  2.991453e+00, -2.162541e+00, ...,
          3.83

In [238]:
data = np.concatenate((X, X_scaled_), axis = 0)

In [239]:
data.shape

(41668, 200, 19)

In [237]:
X_scaled_.shape

(20834, 200, 19)

In [176]:
X_scaled_

array([[[-7.8457201e-01, -3.2197361e+00,  8.6954999e-01, ...,
         -4.6235781e+00, -1.8979239e+00, -8.8516599e-01],
        [ 1.8758599e-01, -1.1729790e+00,  1.4890701e-01, ...,
         -2.1927149e+00, -2.3927770e+00, -3.0271199e-01],
        [ 5.4721200e-01,  2.9876700e-01,  1.9839170e+00, ...,
         -6.2249202e-01,  6.0333939e+00,  6.9941998e-01],
        ...,
        [ 5.9388999e-02,  3.8913000e-02,  2.0692000e-02, ...,
          5.2265000e-02,  1.4797001e-01,  6.0572600e-01],
        [ 8.1523001e-01,  1.8007999e-02,  5.3224301e-01, ...,
          4.0810000e-02,  4.6886399e-01,  5.4632999e-02],
        [ 4.9773999e-02,  3.9177001e-02,  3.3322501e-01, ...,
          1.0725100e-01,  3.8575001e-02,  4.2061001e-02]],

       [[-1.8383899e-01,  5.7446990e+00,  5.8080602e-01, ...,
          1.6859090e+00, -7.8874898e-01,  8.6202002e-01],
        [-6.7979801e-01, -1.8309060e+00,  8.1992298e-01, ...,
          4.8920500e-01, -1.3855720e+00,  3.7590001e+00],
        [ 4.5899501e-01, 

In [151]:
y_= (y_one_hot.values)

In [158]:
y[135]

2.0

In [159]:
y_[135]

array([0, 1, 0, 0, 0], dtype=uint8)

In [161]:
y_.shape

(20834, 5)

In [169]:
seed = 10
y_shuffle = [np.random.shuffle(a) for a in y_]

In [171]:
len(y_shuffle)

20834

In [168]:
for a in y_:
    print(a)

[0 1 0 0 0]
[0 0 0 1 0]
[1 0 0 0 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 0 1 0]
[0 1 0 0 0]
[0 0 1 0 0]
[1 0 0 0 0]
[0 0 0 0 1]
[0 0 0 1 0]
[0 0 1 0 0]
[0 0 0 1 0]
[1 0 0 0 0]
[0 1 0 0 0]
[0 0 0 1 0]
[0 0 1 0 0]
[0 0 0 0 1]
[1 0 0 0 0]
[0 0 0 0 1]
[0 1 0 0 0]
[1 0 0 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 0 1 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 0 0 1 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 1 0 0 0]
[1 0 0 0 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 1 0 0 0]
[0 0 0 0 1]
[0 1 0 0 0]
[1 0 0 0 0]
[0 0 0 1 0]
[0 1 0 0 0]
[0 0 0 0 1]
[0 0 0 1 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 1 0 0 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[1 0 0 0 0]
[0 0 1 0 0]
[1 0 0 0 0]
[1 0 0 0 0]
[0 0 0 0 1]
[0 1 0 0 0]
[1 0 0 0 0]
[0 0 0 0 1]
[0 0 0 1 0]
[1 0 0 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 1 0]
[1 0 0 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 0 0 1]
[1 0

[0 0 1 0 0]
[1 0 0 0 0]
[1 0 0 0 0]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 1 0 0]
[1 0 0 0 0]
[0 0 0 1 0]
[0 1 0 0 0]
[0 0 0 1 0]
[1 0 0 0 0]
[1 0 0 0 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 0 1 0]
[1 0 0 0 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 1 0 0 0]
[0 1 0 0 0]
[1 0 0 0 0]
[0 0 1 0 0]
[1 0 0 0 0]
[1 0 0 0 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 0 1 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 0 0 1]
[0 1 0 0 0]
[0 0 0 0 1]
[1 0 0 0 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 1 0 0]
[1 0 0 0 0]
[1 0 0 0 0]
[0 0 0 0 1]
[1 0 0 0 0]
[1 0 0 0 0]
[1 0 0 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 0 1 0]
[0 0 1 0 0]
[0 1 0 0 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 0 0 1]
[1 0 0 0 0]
[0 0 0 1 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 1 0]
[1 0 0 0 0]
[0 0 1 0 0]
[1 0 0 0 0]
[0 1 0 0 0]
[0 1 0 0 0]
[0 1 0 0 0]
[0 0 0 1 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 1 0 0]
[0 0 0 1 0]
[0 1

[1 0 0 0 0]
[0 0 1 0 0]
[1 0 0 0 0]
[0 1 0 0 0]
[0 0 0 0 1]
[0 1 0 0 0]
[0 1 0 0 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 1 0 0 0]
[0 1 0 0 0]
[1 0 0 0 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 1 0 0 0]
[0 0 1 0 0]
[1 0 0 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 0 1 0 0]
[1 0 0 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[1 0 0 0 0]
[0 0 0 1 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[1 0 0 0 0]
[1 0 0 0 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 1 0 0 0]
[1 0 0 0 0]
[0 0 0 1 0]
[1 0 0 0 0]
[0 0 1 0 0]
[0 1 0 0 0]
[1 0 0 0 0]
[0 0 0 1 0]
[1 0 0 0 0]
[1 0 0 0 0]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 1 0 0]
[1 0 0 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 0 0 1 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 1 0 0 0]
[1 0 0 0 0]
[0 1 0 0 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 0 1 0]
[1 0 0 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 1 0 0 0]
[0 0 0 1 0]
[0 1 0 0 0]
[0 0 1 0 0]
[1 0 0 0 0]
[1 0 0 0 0]
[0 0 0 0 1]
[0 0 0 1 0]
[1 0 0 0 0]
[1 0 0 0 0]
[0 1

[0 0 0 0 1]
[0 1 0 0 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 1 0 0 0]
[0 1 0 0 0]
[0 0 0 1 0]
[1 0 0 0 0]
[0 0 0 0 1]
[0 1 0 0 0]
[0 0 0 1 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[1 0 0 0 0]
[0 0 1 0 0]
[1 0 0 0 0]
[0 0 0 1 0]
[0 0 0 0 1]
[1 0 0 0 0]
[0 1 0 0 0]
[0 1 0 0 0]
[0 1 0 0 0]
[0 0 0 1 0]
[1 0 0 0 0]
[1 0 0 0 0]
[1 0 0 0 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 0 1 0 0]
[0 1 0 0 0]
[1 0 0 0 0]
[1 0 0 0 0]
[0 0 0 0 1]
[0 0 0 1 0]
[0 0 1 0 0]
[0 1 0 0 0]
[0 1 0 0 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 0 1 0]
[0 1 0 0 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 1 0 0 0]
[0 1 0 0 0]
[0 1 0 0 0]
[0 0 0 0 1]
[0 1 0 0 0]
[1 0 0 0 0]
[1 0 0 0 0]
[0 0 0 0 1]
[1 0 0 0 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 1 0 0 0]
[0 0 0 1 0]
[0 1 0 0 0]
[0 0 0 0 1]
[1 0 0 0 0]
[0 0 1 0 0]
[1 0 0 0 0]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 1 0 0]
[1 0 0 0 0]
[1 0 0 0 0]
[0 0 0 0 1]
[0 0 0 1 0]
[0 0 0 0 1]
[0 0 0 1 0]
[0 1 0 0 0]
[0 0 0 0 1]
[0 0

[0 1 0 0 0]
[0 0 0 0 1]
[0 0 0 1 0]
[0 0 0 0 1]
[1 0 0 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 1 0 0 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 1 0 0 0]
[1 0 0 0 0]
[0 0 0 1 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[1 0 0 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 0 1 0]
[0 0 0 1 0]
[1 0 0 0 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 0 0 1 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 1 0 0]
[0 1 0 0 0]
[0 1 0 0 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 0 0 1 0]
[0 1 0 0 0]
[0 1 0 0 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 0 0 0 1]
[0 1 0 0 0]
[0 0 0 1 0]
[0 0 0 1 0]
[1 0 0 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[1 0 0 0 0]
[0 0 0 1 0]
[0 1 0 0 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 0 1 0]
[1 0 0 0 0]
[0 1 0 0 0]
[1 0 0 0 0]
[0 0 0 0 1]
[0 1 0 0 0]
[0 1 0 0 0]
[0 1 0 0 0]
[0 1 0 0 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[1 0 0 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 0 0 1 0]
[0 0 0 1 0]
[1 0 0 0 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 0 0 1]
[1 0 0 0 0]
[1 0 0 0 0]
[0 0 0 0 1]
[0 1 0 0 0]
[0 0

...

[0 0 1 0 0]
[0 1 0 0 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 0 1]
[1 0 0 0 0]
[0 0 0 0 1]
[1 0 0 0 0]
[0 0 0 0 1]
[1 0 0 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[1 0 0 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 0 0 0 1]
[1 0 0 0 0]
[0 0 1 0 0]
[1 0 0 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 0 0 1]
[1 0 0 0 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 1 0 0 0]
[0 0 0 1 0]
[1 0 0 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 0 0 1]
[1 0 0 0 0]
[0 1 0 0 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 0 1 0]
[0 0 1 0 0]
[1 0 0 0 0]
[0 0 0 1 0]
[0 0 1 0 0]
[0 1 0 0 0]
[0 0 0 0 1]
[0 1 0 0 0]
[0 0 0 0 1]
[0 0 0 0 1]
[1 0 0 0 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 0 0 1 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 1 0 0]
[0 0 0 0 1]
[1 0 0 0 0]
[0 0 1 0 0]
[0 1 0 0 0]
[0 0 0 0 1]
[1 0 0 0 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 0 0 0 1]
[0 1 0 0 0]
[0 1 0 0 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 0 0 1 0]
[0 0

[1 0 0 0 0]
[0 0 1 0 0]
[0 1 0 0 0]
[0 1 0 0 0]
[1 0 0 0 0]
[0 0 0 1 0]
[0 0 0 1 0]
[1 0 0 0 0]
[1 0 0 0 0]
[0 0 0 1 0]
[1 0 0 0 0]
[1 0 0 0 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 1 0 0 0]
[0 1 0 0 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 0 0 0 1]
[1 0 0 0 0]
[0 0 0 0 1]
[0 0 0 0 1]
[1 0 0 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[1 0 0 0 0]
[1 0 0 0 0]
[1 0 0 0 0]
[0 0 1 0 0]
[0 1 0 0 0]
[1 0 0 0 0]
[0 1 0 0 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 0 1 0 0]
[0 1 0 0 0]
[0 1 0 0 0]
[0 1 0 0 0]
[1 0 0 0 0]
[0 0 0 1 0]
[0 1 0 0 0]
[0 0 0 0 1]
[0 1 0 0 0]
[0 0 0 0 1]
[1 0 0 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 0 1 0]
[0 0 0 1 0]
[1 0 0 0 0]
[0 0 0 1 0]
[1 0 0 0 0]
[0 0 0 0 1]
[1 0 0 0 0]
[1 0 0 0 0]
[0 0 0 0 1]
[1 0 0 0 0]
[0 1 0 0 0]
[0 0 0 1 0]
[0 1 0 0 0]
[1 0 0 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[1 0 0 0 0]
[0 0 0 1 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 0 1 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 0 0 0 1]
[1 0 0 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[1 0 0 0 0]
[0 1 0 0 0]
[1 0 0 0 0]
[0 0 0 1 0]
[1 0 0 0 0]
[0 0 0 0 1]
[0 0 0 1 0]
[0 0 1 0 0]
[0 0

[0 0 0 1 0]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 0 1 0]
[0 1 0 0 0]
[1 0 0 0 0]
[0 0 0 0 1]
[0 1 0 0 0]
[1 0 0 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 1 0 0 0]
[0 1 0 0 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 1 0 0 0]
[1 0 0 0 0]
[0 0 0 1 0]
[0 1 0 0 0]
[0 1 0 0 0]
[1 0 0 0 0]
[1 0 0 0 0]
[1 0 0 0 0]
[0 0 0 0 1]
[0 0 0 1 0]
[0 0 0 1 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 0 0 1]
[1 0 0 0 0]
[1 0 0 0 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 0 1 0 0]
[1 0 0 0 0]
[0 1 0 0 0]
[0 0 0 0 1]
[0 0 0 1 0]
[0 0 0 1 0]
[0 1 0 0 0]
[1 0 0 0 0]
[0 0 1 0 0]
[1 0 0 0 0]
[1 0 0 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[1 0 0 0 0]
[0 1 0 0 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 0 1 0 0]
[0 1 0 0 0]
[0 0 0 1 0]
[0 1 0 0 0]
[1 0 0 0 0]
[0 1 0 0 0]
[0 0 0 1 0]
[1 0 0 0 0]
[0 1 0 0 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 0 0 1 0]
[0 1 0 0 0]
[1 0 0 0 0]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 1 0 0]
[0 0

[0 0 0 0 1]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 1 0 0 0]
[0 1 0 0 0]
[1 0 0 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[1 0 0 0 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 1 0 0]
[1 0 0 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 1 0 0 0]
[0 0 0 1 0]
[0 0 0 0 1]
[1 0 0 0 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 1 0 0 0]
[0 1 0 0 0]
[0 1 0 0 0]
[0 0 0 1 0]
[0 0 1 0 0]
[0 1 0 0 0]
[0 1 0 0 0]
[0 0 0 0 1]
[0 1 0 0 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 1 0 0 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 0 0 1 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 1 0 0 0]
[0 0 0 1 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 1 0 0 0]
[1 0 0 0 0]
[0 0 0 1 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 1 0 0 0]
[1 0 0 0 0]
[0 0 1 0 0]
[0 1 0 0 0]
[0 1 0 0 0]
[0 1 0 0 0]
[0 0 0 0 1]
[0 0 0 0 1]
[0 0 0 1 0]
[0 1 0 0 0]
[0 0 0 1 0]
[0 0 1 0 0]
[1 0 0 0 0]
[0 1 0 0 0]
[0 0 0 1 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[1 0 0 0 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 1 0 0 0]
[1 0 0 0 0]
[0 0 0 0 1]
[0 1

[0 0 1 0 0]
[0 0 0 1 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 0 1 0]
[0 0 0 0 1]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 0 1 0]
[1 0 0 0 0]
[0 1 0 0 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[1 0 0 0 0]
[0 0 0 1 0]
[0 0 0 1 0]
[0 1 0 0 0]
[0 0 0 1 0]
[1 0 0 0 0]
[0 0 0 1 0]
[0 0 1 0 0]
[0 0 0 0 1]
[1 0 0 0 0]
[0 0 0 0 1]
[0 0 1 0 0]
[1 0 0 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[0 1 0 0 0]
[0 0 0 0 1]
[0 0 0 1 0]
[1 0 0 0 0]
[0 0 0 0 1]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 1 0 0]
[0 1 0 0 0]
[0 1 0 0 0]
[1 0 0 0 0]
[1 0 0 0 0]
[0 1 0 0 0]
[0 1 0 0 0]
[1 0 0 0 0]
[0 0 0 1 0]
[0 0 1 0 0]
[0 0 0 0 1]
[0 0 1 0 0]
[0 0 0 0 1]
[1 0 0 0 0]
[0 1 0 0 0]
[0 0 0 0 1]
[0 1 0 0 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 1 0 0 0]
[0 0 1 0 0]
[0 0 1 0 0]
[1 0 0 0 0]
[0 0 0 0 1]
[0 0 0 1 0]
[0 1 0 0 0]
[0 0 0 1 0]
[0 0 0 1 0]
[1 0 0 0 0]
[1 0 0 0 0]
[1 0 0 0 0]
[0 0 1 0 0]
[0 1 0 0 0]
[0 1 0 0 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 0 1 0 0]
[1 0 0 0 0]
[0 0

# Model

In [286]:
input_ = Input(shape = (200,19))
x = LSTM(200, return_sequences=True)(input_)
x = Dropout(0.3)(x)
x = LSTM(200, return_sequences= True)(x)
x = LSTM(100)(x)
x = Dropout(0.2)(x)
x = Dense(units = 100, activation='relu')(x)
output = Dense(8,  activation='softmax')(x)

In [287]:
Stacked_LSTM_model = Model(input_, output)

In [283]:
Stacked_LSTM_model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 200, 19)           0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 200, 200)          176000    
_________________________________________________________________
dropout_9 (Dropout)          (None, 200, 200)          0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 200, 200)          320800    
_________________________________________________________________
lstm_24 (LSTM)               (None, 100)               120400    
_________________________________________________________________
dropout_10 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 100)               1010

In [268]:
from keras.utils import plot_model

In [270]:
plot_model(Stacked_LSTM_model)

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.